# Practica 1 - Landscapes
This dataset represents different countries or inland areas with 14 different classes. Due to their difficulty, the images are real images without any practical pre-treatment, it will be necessary to maximise the characteristics of the images to improve the results.

## Transforming and preparing the dataset

The first problem we find is that images don't have the same dimensions, so we have to scale all the images to the same size in order to train our model. The best size is the minimum of all images, in order to avoid useless pixels of information because if we increase the size of the small images we are causing pixels that didn't exist to appear and alter the results.

In [1]:
%pip install opencv-python

  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from PIL import Image
import os

def find_smallest_image_in_directory(directory):
    image_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg')]

    if not image_files:
        print(f"No images found in the directory {directory}.")
        return None, None

    min_size = float('inf')
    min_image = None

    for filename in image_files:
        image = Image.open(os.path.join(directory, filename))
        width, height = image.size
        current_size = width * height

        if current_size < min_size:
            min_size = current_size
            min_image = filename

    return min_image, min_size

base_directory = "dat/a2/a2/data/train"

# Iterate through subdirectories
for subdirectory in os.listdir(base_directory):
    subdirectory_path = os.path.join(base_directory, subdirectory)

    if os.path.isdir(subdirectory_path):
        min_image, min_size = find_smallest_image_in_directory(subdirectory_path)

        if min_image is not None:
            print(f"Found in {subdirectory}: {min_image}")
            print(f"Dimensions: {Image.open(os.path.join(subdirectory_path, min_image)).size}")
        else:
            print(f"No images found in {subdirectory}.")



Found in bedroom: image_0038.jpg
Dimensions: (224, 200)
Found in Coast: image_0006.jpg
Dimensions: (256, 256)
Found in Forest: image_0003.jpg
Dimensions: (256, 256)
Found in Highway: image_0009.jpg
Dimensions: (256, 256)
Found in industrial: image_0019.jpg
Dimensions: (222, 220)
Found in Insidecity: image_0005.jpg
Dimensions: (256, 256)
Found in kitchen: image_0202.jpg
Dimensions: (203, 220)
Found in livingroom: image_0253.jpg
Dimensions: (220, 224)
Found in Mountain: image_0002.jpg
Dimensions: (256, 256)
Found in Office: image_0014.jpg
Dimensions: (241, 220)
Found in OpenCountry: image_0003.jpg
Dimensions: (256, 256)
Found in store: image_0054.jpg
Dimensions: (221, 220)
Found in Street: image_0001.jpg
Dimensions: (256, 256)
Found in Suburb: image_0002.jpg
Dimensions: (330, 220)
Found in TallBuilding: image_0010.jpg
Dimensions: (256, 256)


As we can see the minimum size is (203, 200). To keep things simple we can escale all the images to a square of size (200,200) 

In [3]:

import os
import cv2
import IPython.display as display
from PIL import Image

def resize_and_display_images_in_directory(directory, target_size=(200, 200), num_images=5):
    image_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg')]

    if not image_files:
        print(f"No images found in the directory {directory}.")
        return
    
    # Create a subdirectory to store modified images
    resized_directory = os.path.join(directory, "resized")
    os.makedirs(resized_directory, exist_ok=True)

    # Display up to num_images images
    for filename in image_files[:num_images]:
        image_path = os.path.join(directory, filename)
        original_image = Image.open(image_path)

        # Resize the image
        resized_image = original_image.resize(target_size)
        
        # Save the resized image back to the same directory
        resized_image.save(os.path.join(resized_directory, filename))

        # Display the resized image in the notebook
        #display.display(resized_image)

# Directory path
base_directory = "dat/a2/a2/data/train"

# Iterate through subdirectories
for subdirectory in os.listdir(base_directory):
    subdirectory_path = os.path.join(base_directory, subdirectory)

    if os.path.isdir(subdirectory_path):
        num_total_images = len([file for file in os.listdir(subdirectory_path) if file.lower().endswith('.jpg')])
        # Resize and display up to 5 images from each directory
        resize_and_display_images_in_directory(subdirectory_path, num_images=num_total_images)


In [4]:
import os
import cv2
import numpy as np

# Función para extraer características HOG de una imagen
def extract_hog_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    win_size = (64, 64)
    cell_size = (8, 8)
    block_size = (16, 16)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size, cell_size, cell_size, nbins)
    features = hog.compute(image)
    return features

# Directorio base
base_directory = "dat/a2/a2/data/train/bedroom/resized"

# Lista para almacenar características
features_list = []

# Iterar a través de los subdirectorios
for subdirectory in os.listdir(base_directory):
    subdirectory_path = os.path.join(base_directory, subdirectory)

    if os.path.isdir(subdirectory_path):
        num_total_images = len([file for file in os.listdir(subdirectory_path) if file.lower().endswith('.jpg')])

        # Redimensionar y procesar todas las imágenes en cada directorio
        resize_and_display_images_in_directory(subdirectory_path, num_images=num_total_images)

        # Extraer características de cada imagen
        for filename in os.listdir(subdirectory_path):
            if filename.lower().endswith('.jpg'):
                image_path = os.path.join(subdirectory_path, filename)
                features = extract_hog_features(image_path)
                features_list.append(features)
